# QR Algorithm

Ci sono delle proprietà:

- Dato che gli eigenvalue di una matrice tirangolare sono gli elementi sulla diagonale, questo algoritmo permette di trovarli.
- Similarity transformation preserva gli eigenvalue.

La matrice A viene **decomposta** in A = QR dove Q è ortogonale e R è triangolare superiore.

In [2]:
import numpy as np
from itertools import product
from math import copysign, hypot, sqrt
from scipy.linalg import hessenberg

def print_matrix(matrix):
    s = [[str(e) for e in row] for row in matrix]
    lens = [max(map(len, col)) for col in zip(*s)]
    fmt = '\t'.join('{{:{}}}'.format(x) for x in lens)
    table = [fmt.format(*row) for row in s]
    print('\n'.join(table))
    return

## Gram-Schmidt
Come prima implementazione proviamo con l'algoritmo di **Gram-Schmidt**


In [ ]:
def qr_decomp_gram_schmidt(A):
    m, n = A.shape
    rank = np.linalg.matrix_rank(A)

    if rank < n:
        print(f"Il rango della matrice è {rank} il quale è inferiore del numero delle colonne {n}!")
    
    Q = np.zeros((m, n))
    
    for i, column in enumerate(A.T):
        Q[:,i] = column

        for prev in Q.T[:i]:
            Q[:,i] -= (prev @ column) / (prev @ prev) * prev
    
    Q /= np.linalg.norm(Q,axis=0)
    R = Q.T @ A

    return Q, R

In [ ]:
# Tests

#TODO: rendere i test non dipendenti da una sola funzione

def test_orthonormality(A):
    Q, _ = qr_decomp_gram_schmidt(A)
    I = Q.T @ Q

    assert len(set(I.shape)) == 1, "L'identità computata deve essere una matrice quadrata!"
    assert np.allclose(I, np.eye(I.shape[0])), "L'identità  computata deve essere una matrice identità!"

def test_upper_triangular(A):
    _, R = qr_decomp_gram_schmidt(A)

    assert len(set(R.shape)) == 1, "La matrice triangolare superiore computata dovrebbe essere una matrice quadrata!"

    for i, row in enumerate(R):
        assert np.allclose(row[:i], 0), f"La riga {i} della matrice triangolare superiore calcolata non è giusta!"

def test_multiplication(A):
    Q,R = qr_decomp_gram_schmidt(A)
    
    assert np.allclose(Q @ R, A), "La moltiplicazione fra le matrici ottenute non restituisce la matrice originaria!"

In [ ]:
test_matricies = (
    np.random.random((2, 2)),
    np.random.random((5, 2)),
    np.random.random((5, 5)),
    np.random.random((9, 8)),
)

tests = (
    test_orthonormality,
    test_upper_triangular,
    test_multiplication,
)

for test, matrix in product(tests, test_matricies):
    test(matrix)

print("tutti i test sono superati")

# Givens Rotation
proviamo ad implementare le givens rotation per migliorare la velocità di esecuzione.

In [ ]:
def qr_decomp_givens_rotation(A):

    (num_rows, num_cols) = np.shape(A)
    if num_rows == num_cols: 
        # Initialize Q,R
        # Q = orthogonal matrix
        # R =  upper triangular matrix
        Q = np.identity(num_rows)
        R = np.copy(A)

        # Iterate over lower triangular matrix.
        (rows, cols) = np.tril_indices(num_rows, -1, num_cols)
        #print(rows, cols)
        for (row, col) in zip(rows, cols):

            # Compute Givens rotation matrix and
            # zero-out lower triangular matrix entries.
            if R[row, col] != 0:
                (c, s) = Givens_Rotation_Matrix_Entries(R[col, col], R[row, col])

                G = np.identity(num_rows)
                G[[col, row], [col, row]] = c
                G[row, col] = s
                G[col, row] = -s

                R = np.dot(G, R)
                Q = np.dot(Q, G.T)
                #R = G @ R
                #Q = Q @ G.T
        return Q, R
    else:
        Q = np.eye(num_rows)
        R = np.copy(A)

        rows, cols = np.tril_indices(num_rows, -1, num_cols)
        for (row, col) in zip(rows, cols):
            # If the subdiagonal element is nonzero, then compute the nonzero 
            # components of the rotation matrix
            if R[row, col] != 0:
                r = np.sqrt(R[col, col]**2 + R[row, col]**2)
                c, s = R[col, col]/r, -R[row, col]/r

                # The rotation matrix is highly discharged, so it makes no sense 
                # to calculate the total matrix product
                R[col], R[row] = R[col]*c + R[row]*(-s), R[col]*s + R[row]*c
                Q[:, col], Q[:, row] = Q[:, col]*c + Q[:, row]*(-s), Q[:, col]*s + Q[:, row]*c

        return Q[:, :num_cols], R[:num_cols]



##### Compute matrix entries for Givens rotation. #####

def Givens_Rotation_Matrix_Entries(a, b):
    r = hypot(a, b)
    c = a/r
    s = -b/r

    return (c, s)

In [ ]:
def test_orthonormality(A):
    Q, _ = qr_decomp_givens_rotation(A)
    I = Q.T @ Q

    assert len(set(I.shape)) == 1, "L'identità computata deve essere una matrice quadrata!"
    assert np.allclose(I, np.eye(I.shape[0])), "L'identità  computata deve essere una matrice identità!"

def test_upper_triangular(A):
    _, R = qr_decomp_givens_rotation(A)

    assert len(set(R.shape)) == 1, "La matrice triangolare superiore computata dovrebbe essere una matrice quadrata!"

    for i, row in enumerate(R):
        assert np.allclose(row[:i], 0), f"La riga {i} della matrice triangolare superiore calcolata non è giusta!"

def test_multiplication(A):
    Q, R = qr_decomp_givens_rotation(A)
    
    assert np.allclose(Q @ R, A), "La moltiplicazione fra le matrici ottenute non restituisce la matrice originaria!"

In [ ]:
test_matricies = (
    np.random.random((3, 3)),
    np.random.random((4, 5)),
    np.random.random((5, 5)),
    np.random.random((9, 8)),
)

tests = (
    test_orthonormality,
    test_multiplication,
)

for test, matrix in product(tests, test_matricies):
    test(matrix)

print("tutti i test sono superati")

In [ ]:
A = np.random.random((4, 4))

Q, R = qr_decomp_givens_rotation(A)

print(A)
print(Q)
print(R)

print(Q @ R)

# Shifted QR Algorithm

Questo approccio ci consente di ridurre considerevolmente il tempo di esecuzione dell'algoritmo QR, ma per poterlo utilizzare è necessario ridurre la matrice di partenza in forma di Hessenberg

## Riduzione in forma di Hessenberg
Per ridurre la matrice di partenza A in forma di Hessenberg utilizzeremo le Givens Rotation, per azzerare gli elementi sotto la prima sottodiagonale.
Iteriamo lungo le colonne a partire dall'ultimo elemento, così da poter scegliere come pivot l'elemento immediatamente sopra.

In [88]:
# TODO da controllare se funziona per matrici non quadrate, e in caso implementare una versione funzionante.

def Givens_Rotation_Matrix_Entries(a, b):
    #r = hypot(a.astype(complex), b.astype(complex))
    r = sqrt(a.astype(complex)**2 + b.astype(complex)**2)
    c = a/r
    s = b/r    
    return c.astype(complex), s.astype(complex)

def Hessenberg_reduction(A):
    # mi trovo gli indici della sottodiagonale corrispondente agli elementi da azzerarcole
    (num_rows, num_cols) = np.shape(A)
    # Q ci serve per calcolarci la forma di Schur
    Q = np.identity(num_rows)
    Q.astype(complex)
    #itero sugli indici appena trovati 
    for col in range(num_cols-2):
        for row in reversed(range(2+col,num_cols)):
            # trova gli indici della matrice di givens unando come pivot [col, col](sempre != 0)
            c, s = Givens_Rotation_Matrix_Entries(A[row-1, col], A[row, col])
            #si costruisce la matrice di givens partendo dall'identità
            G = np.identity(num_rows)
            G.astype(complex)
            #assegna gli indici trovati precedentemente alla matrice di givens
            G[row-1, row-1] = c
            G[row, row] = c
            G[row, row-1] = -s
            G[row-1, row] = s
            # annichilisce l'elemento in posizione A[row, col]
            A = np.dot(G,A)
            Q = np.dot(Q, G.T)
            if A[row, col] < 1.e-16:
                A[row,col] = 0

    return A, Q

In [49]:
A = np.random.random((5, 5))
(A, Q) = Hessenberg_reduction(A)
print_matrix(A)

0.4565091508729885	0.28372194992958155	0.7057243411318634	0.6623844629830221 	0.3760065700653733 
1.1914450560218222	1.0397780321382748 	0.8061908575352016	1.108255415433371  	0.7524173985732907 
0.0               	0.6608746151837372 	0.6580084244527941	0.915773059028672  	0.5257493902486314 
0.0               	0.0                	0.5580932589692382	0.11854930466217825	-0.5939833919711694
0.0               	0.0                	0.0               	0.24434448197975886	-0.1857717890938433


C:\Users\tomma\AppData\Local\Temp\ipykernel_12632\1682300848.py:24: ComplexWarning: Casting complex values to real discards the imaginary part
  G[row-1, row-1] = c
C:\Users\tomma\AppData\Local\Temp\ipykernel_12632\1682300848.py:25: ComplexWarning: Casting complex values to real discards the imaginary part
  G[row, row] = c
C:\Users\tomma\AppData\Local\Temp\ipykernel_12632\1682300848.py:26: ComplexWarning: Casting complex values to real discards the imaginary part
  G[row, row-1] = -s
C:\Users\tomma\AppData\Local\Temp\ipykernel_12632\1682300848.py:27: ComplexWarning: Casting complex values to real discards the imaginary part
  G[row-1, row] = s


## Shifted QR
Ora che abbiamo una matrice in forma di Hessenberg possiamo passare all'implementazione dello Shifted QR algorithm, l'aumento di velocità dipende dal fatto che, se si parte da una matrice in forma di Hessenberg allora sono necessarie solo O(n^2) operazioni.

In particolare noi utilizzeremo una single shift strategy con i Rayleigh quotient shift

In [84]:
def shifted_QR(A):
    H, _ = Hessenberg_reduction(A)
    #H = hessenberg(A)
    H.astype(complex)
    (num_rows, num_cols) = np.shape(H)
    for m in range(num_rows-1, 0, -1):
        count = 0
        while (abs(H[m,m-1]) > 1.e-20) and 1000 > count:
            count += 1
            Q, R = np.linalg.qr(H - (H[m,m] * np.eye(num_rows)))
            H = (np.dot(R, Q)) + (H[m,m] * np.eye(num_rows))
        H[m,m-1] = 0
    return H

In [92]:
A = np.random.random((5, 5))
print(np.linalg.eig(A)[0])
A = A.astype(complex)
H = shifted_QR(A)
print_matrix(H)

[ 3.36719471+0.j         -0.20301719+0.33937993j -0.20301719-0.33937993j
 -0.1939922 +0.j          0.64398842+0.j        ]
(2.652633625860788+0j)	(0.18849520784844043+0j)	(-1.1383656587803987+0j) 	(-0.41084409149079704+0j)	(1.647547775686951+0j)    
0j                    	(0.1714924893351991+0j) 	(0.2783229973442452+0j)  	(0.5643934401984574+0j)  	(-0.061822495961350796+0j)
0j                    	0j                      	(-0.18305910123860694+0j)	(-0.41217222724002106+0j)	(0.043147115789331315+0j) 
0j                    	0j                      	0j                       	(-0.3419512037384712+0j) 	(-0.3155198186628245+0j)  
0j                    	0j                      	0j                       	0j                       	(0.7770178075358893+0j)   


C:\Users\tomma\AppData\Local\Temp\ipykernel_12632\4006317831.py:5: ComplexWarning: Casting complex values to real discards the imaginary part
  r = sqrt(a.astype(complex)**2 + b.astype(complex)**2)
C:\Users\tomma\AppData\Local\Temp\ipykernel_12632\4006317831.py:25: ComplexWarning: Casting complex values to real discards the imaginary part
  G[row-1, row-1] = c
C:\Users\tomma\AppData\Local\Temp\ipykernel_12632\4006317831.py:26: ComplexWarning: Casting complex values to real discards the imaginary part
  G[row, row] = c
C:\Users\tomma\AppData\Local\Temp\ipykernel_12632\4006317831.py:27: ComplexWarning: Casting complex values to real discards the imaginary part
  G[row, row-1] = -s
C:\Users\tomma\AppData\Local\Temp\ipykernel_12632\4006317831.py:28: ComplexWarning: Casting complex values to real discards the imaginary part
  G[row-1, row] = s
